In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.utils.vis_utils import plot_model
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten, LSTM
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
# data is being refined, Here we fist get our data to be converted into pandas dataframae
# then we drop empty spaces and eradicate useless indices and getting data in float type
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/MyDrive/Code/dataset/IoT-BoT.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
data = cleaningdata(dataset)

# collecting all required colomns in data to X
X = data.iloc[:, 0:85].values

# collecting our LABEL colomn in y
y = data.iloc[:, -2].values

from sklearn.preprocessing import label_binarize

Y = label_binarize(y, classes=[0, 1, 2, 3, 4])
n_classes = Y.shape[1]
print(n_classes)

5


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from keras.utils.np_utils import to_categorical

for train, test in kfold.split(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    x_train = np.array(X_train)
    x_test = np.array(X_test)
    y_train1 = np.array(y_train)
    y_test1 = np.array(y_test)

    y_train= to_categorical(y_train1)
    y_test= to_categorical(y_test1)

    model = Sequential()
    model.add(LSTM(16,input_dim=85, return_sequences=True))  
    model.add(Dropout(0.6))
    model.add(LSTM(16, return_sequences=True))  
    model.add(Dropout(0.6))
    model.add(LSTM(16, return_sequences=False)) 
    model.add(Dropout(0.6))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


Epoch 1/10
24255/24255 [==============================] - 92s 4ms/step - loss: 0.1068 - accuracy: 0.9656 - mae: 0.0251
Epoch 2/10
24255/24255 [==============================] - 89s 4ms/step - loss: 0.0462 - accuracy: 0.9886 - mae: 0.0097
Epoch 3/10
24255/24255 [==============================] - 87s 4ms/step - loss: 0.0411 - accuracy: 0.9907 - mae: 0.0081
Epoch 4/10
24255/24255 [==============================] - 89s 4ms/step - loss: 0.0372 - accuracy: 0.9913 - mae: 0.0070
Epoch 5/10
24255/24255 [==============================] - 88s 4ms/step - loss: 0.0347 - accuracy: 0.9922 - mae: 0.0065
Epoch 6/10
24255/24255 [==============================] - 90s 4ms/step - loss: 0.0338 - accuracy: 0.9928 - mae: 0.0062
Epoch 7/10
24255/24255 [==============================] - 92s 4ms/step - loss: 0.0328 - accuracy: 0.9921 - mae: 0.0060
Epoch 8/10
24255/24255 [==============================] - 90s 4ms/step - loss: 0.0348 - accuracy: 0.9927 - mae: 0.0061
Epoch 9/10
24255/24255 [========================

In [ ]:
target_names = ['DoS', 'DDOS', 'Reconnaissance', 'Normal','Theft']

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00    509191
          DDOS       1.00      1.00      1.00    520809
Reconnaissance       1.00      1.00      1.00     77702
        Normal       1.00      1.00      1.00    444203
         Theft       0.00      0.00      0.00       406

     micro avg       1.00      1.00      1.00   1552311
     macro avg       0.80      0.80      0.80   1552311
  weighted avg       1.00      1.00      1.00   1552311
   samples avg       1.00      1.00      1.00   1552311

Accuracy_Train: 0.9996901394114968


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision_Train: 0.9994288439320177
Recall_Train: 0.9996901394114968
f1-score_Train: 0.9995594321342535
ROC_AUC_Train: 0.9999946998055369


In [ ]:
### test_accuracy
import sklearn.metrics as metrics
expected_test = y_test
predicted_test = model.predict(X_test)
data = classification_report(expected_test,np.round(predicted_test),target_names=target_names)
print(data)

print("Accuracy:",metrics.accuracy_score(expected_test, np.round(predicted_test)))
print("Precision:",metrics.precision_score(expected_test, np.round(predicted_test), average='weighted'))
print("Recall:",metrics.recall_score(expected_test, np.round(predicted_test), average='weighted'))
print("f1-score:",metrics.f1_score(expected_test, np.round(predicted_test), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected_test, predicted_test,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00    127348
          DDOS       1.00      1.00      1.00    130313
Reconnaissance       1.00      1.00      1.00     19495
        Normal       1.00      1.00      1.00    110808
         Theft       0.00      0.00      0.00       114

     micro avg       1.00      1.00      1.00    388078
     macro avg       0.80      0.80      0.80    388078
  weighted avg       1.00      1.00      1.00    388078
   samples avg       1.00      1.00      1.00    388078

Accuracy: 0.9996521317879395


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9993586794735433
Recall: 0.9996521317879395
f1-score: 0.99950533657904
ROC_AUC: 0.9999945549305275
